## Analyzing the impact of weather on bike trip duration using OLS regression method



In [1]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 999)

# All Libraries needed for Regression imported

# All Dataframes as CSV needed for regression

bike_journey_data=pd.read_csv('bike_journey_data_27_mar_2020.csv')
weather_data=pd.read_csv('weather_data_27_mar_2020.csv')

In [2]:
'''weather_data=weather_data.drop(columns=['Date','Date Converted','Hours','id'])
weather_data=weather_data.set_index('id_Hours')

from scipy import stats
import numpy as np

df = pd.DataFrame(weather_data)
df.head()

z = np.abs(stats.zscore(df))
print(z)
threshold = 4
print(np.where(z > 4))

df = df[(z < 4).all(axis=1)]
print(df.shape)'''

"weather_data=weather_data.drop(columns=['Date','Date Converted','Hours','id'])\nweather_data=weather_data.set_index('id_Hours')\n\nfrom scipy import stats\nimport numpy as np\n\ndf = pd.DataFrame(weather_data)\ndf.head()\n\nz = np.abs(stats.zscore(df))\nprint(z)\nthreshold = 4\nprint(np.where(z > 4))\n\ndf = df[(z < 4).all(axis=1)]\nprint(df.shape)"

In [3]:
'''x=df['precipIntensity']
plt.boxplot(x, notch=None)'''

"x=df['precipIntensity']\nplt.boxplot(x, notch=None)"

In [4]:
'''statistics = df['precipIntensity'].describe()
statistics'''

"statistics = df['precipIntensity'].describe()\nstatistics"

In [5]:
# Collpase the bike and weather dataset by trips by hour and sum the number of trips and total duration of all trips by hour
reg_merge= bike_journey_data.merge(right=weather_data,
                             left_on = 'id_hours',
                             right_on = 'id_Hours')

reg_merge['bike_trips_by_hour'] = 1
def bike_trips_by_hour(reg_merge, renamed_count):
    bike_trips_by_hour = reg_merge.loc[:, ('id_hours', 'bike_trips_by_hour')].groupby(['id_hours']).sum()
    bike_trips_by_hour.reset_index(inplace=True)
    bike_trips_by_hour.rename(columns={'bike_trips_by_hour': str(renamed_count)}, inplace=True)
    return bike_trips_by_hour
bike_trips_by_hour = bike_trips_by_hour(reg_merge, 'bike_trips_by_hour')

reg_merge3= reg_merge.merge(right=bike_trips_by_hour,
                             left_on = 'id_hours',
                             right_on = 'id_hours')

reg_merge3['Duration in hours'] = 1
def duration_by_hour(reg_merge3, renamed_count):
    duration_by_hour = reg_merge3.loc[:, ('id_hours', 'Duration in minutes')].groupby(['id_hours']).sum()
    duration_by_hour.reset_index(inplace=True)
    duration_by_hour.rename(columns={'Duration in minutes': str(renamed_count)}, inplace=True)
    return duration_by_hour
duration_by_hour = duration_by_hour(reg_merge3, 'duration_by_hour')

regression_dataset= reg_merge3.merge(right=duration_by_hour,
                             left_on = 'id_hours',
                             right_on = 'id_hours')

regression_dataset['Avg trip duration']=regression_dataset['duration_by_hour']/regression_dataset['bike_trips_by_hour_y']

In [6]:
df_month=pd.get_dummies(regression_dataset.Month)
df_month.columns=['Jan','Feb','Mar', 'Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
#df_month=df_month.astype(int)
regression_dataset = regression_dataset.join(df_month)

In [7]:
##Generating dummy variables based on temperature

regression_dataset["temp_0"] = regression_dataset['temperature'].apply(lambda x: 1 if x <=0 else 0)
regression_dataset["temp_5"] = regression_dataset['temperature'].apply(lambda x: 1 if x>0 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=5 else 0)
regression_dataset["temp_10"] = regression_dataset['temperature'].apply(lambda x: 1 if x>5 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=10 else 0)
regression_dataset["temp_15"] = regression_dataset['temperature'].apply(lambda x: 1 if x>10 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=15 else 0)

regression_dataset["temp_20"] = regression_dataset['temperature'].apply(lambda x: 1 if x>15 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=20 else 0)
regression_dataset["temp_25"] = regression_dataset['temperature'].apply(lambda x: 1 if x>20 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=25 else 0)

regression_dataset["temp_30"] = regression_dataset['temperature'].apply(lambda x: 1 if x>25 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=30 else 0)
regression_dataset["temp_35"] = regression_dataset['temperature'].apply(lambda x: 1 if x>30 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=35 else 0)
regression_dataset["temp_40"] = regression_dataset['temperature'].apply(lambda x: 1 if x>35 else 0)
regression_dataset['Jan']=regression_dataset['Jan'].replace(1,0)


In [8]:
##Generating a dummy variables for humidity and rain based on the mean value (0.6497)
regression_dataset["relative_humidity"] = regression_dataset['humidity'].apply(lambda x: 1 if x >=0.6497 else 0)

regression_dataset["rain"] = regression_dataset['precipIntensity'].apply(lambda x: 1 if x >0.0697 else 0)

In [9]:
#x=regression_dataset['humidity']
#plt.boxplot(x, notch=None)

In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

y = 'Avg trip duration'
x_1 = ['temp_0','temp_5', 'temp_10', 'temp_15', 'temp_20', 'temp_25', 'temp_30', 'temp_35']
x_2 = ['temp_0','temp_5', 'temp_10', 'temp_15', 'temp_20', 'temp_25', 'temp_30', 'temp_35','relative_humidity', 'rain','Feb','Mar', 'Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
x_3 = ['temp_0','temp_5', 'temp_10', 'temp_15', 'temp_20', 'temp_25', 'temp_30', 'temp_35','humidity','precipIntensity','Feb','Mar', 'Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']

#model_Simple = sm.OLS(temperature_10_dataset[y], temperature_10_dataset[x]).fit()

model_1 = sm.OLS(regression_dataset[y], sm.add_constant(regression_dataset[x_1])).fit()
model_1.summary()
model_2 = sm.OLS(regression_dataset[y], sm.add_constant(regression_dataset[x_2])).fit()
model_3 = sm.OLS(regression_dataset[y], sm.add_constant(regression_dataset[x_3])).fit()

/Users/albegjonbalaj/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [11]:
dfoutput = summary_col([model_1, model_2, model_3],stars=True)
print(dfoutput)


                  Avg trip duration I Avg trip duration II Avg trip duration III
--------------------------------------------------------------------------------
Apr                                   1.2335***            0.9435***            
                                      (0.0116)             (0.0115)             
Aug                                   -1.9714***           -1.3144***           
                                      (0.0129)             (0.0128)             
Dec                                   2.5049***            2.7019***            
                                      (0.0113)             (0.0113)             
Feb                                   0.6096***            0.8480***            
                                      (0.0112)             (0.0111)             
Jul                                   -3.4095***           -2.5703***           
                                      (0.0129)             (0.0129)             
Jun                        